In [185]:
import numpy as np
import pandas as pd

#### Load the matrices

In [186]:
battery_expenditure = pd.read_csv('battery_expenditure_matrix.csv')
distance_matrix = pd.read_csv('distance_matrix.csv')
time_matrix = pd.read_csv('time_matrix.csv')

#### Generating random combination

In [187]:
num_sequences=20
start_node=0
end_node=16
battery_threshold=10

w1=0.5
w2=0.5

In [188]:
def generate_random_sequences(num_sequences, battery_threshold=25):
    sequences = []
    node_list = list(range(17))

    for _ in range(num_sequences):
        while True:
            shuffled_nodes = np.random.permutation(node_list[1:-1])
            sequence = [start_node] + shuffled_nodes.tolist() + [end_node]
            battery_level = 78
            valid_sequence = True
            
            for i in range(len(sequence) - 1):
                node1 = sequence[i]
                node2 = sequence[i + 1]
                battery_consumption = battery_expenditure.iloc[node1, node2]
                battery_level -= battery_consumption
                if battery_level < battery_threshold:
                    valid_sequence = False
                    break
            
            if valid_sequence:
                sequences.append(sequence)
                break

    return sequences

In [189]:
random_seq=generate_random_sequences(num_sequences)
random_seq

[[0, 2, 8, 13, 3, 10, 9, 5, 6, 15, 12, 1, 14, 4, 7, 11, 16],
 [0, 4, 9, 11, 3, 6, 2, 10, 14, 1, 5, 12, 15, 13, 7, 8, 16],
 [0, 1, 11, 7, 6, 9, 4, 2, 8, 13, 5, 3, 12, 15, 10, 14, 16],
 [0, 2, 13, 7, 11, 6, 15, 1, 8, 4, 5, 10, 9, 12, 3, 14, 16],
 [0, 7, 15, 14, 5, 2, 1, 12, 9, 4, 10, 13, 6, 3, 8, 11, 16],
 [0, 4, 11, 2, 8, 14, 1, 13, 9, 5, 12, 3, 6, 15, 10, 7, 16],
 [0, 11, 10, 4, 1, 13, 6, 2, 7, 8, 15, 12, 3, 14, 5, 9, 16],
 [0, 6, 7, 4, 10, 14, 12, 1, 5, 2, 9, 8, 13, 3, 15, 11, 16],
 [0, 11, 8, 5, 12, 1, 2, 15, 6, 9, 3, 14, 4, 10, 7, 13, 16],
 [0, 15, 13, 6, 7, 8, 14, 12, 9, 3, 4, 2, 5, 11, 1, 10, 16],
 [0, 13, 3, 2, 12, 9, 11, 7, 5, 6, 8, 15, 1, 14, 4, 10, 16],
 [0, 12, 15, 13, 7, 14, 3, 5, 2, 10, 9, 8, 11, 1, 4, 6, 16],
 [0, 5, 9, 10, 2, 4, 3, 8, 11, 1, 13, 14, 6, 15, 7, 12, 16],
 [0, 12, 3, 8, 14, 11, 6, 13, 7, 2, 4, 15, 5, 10, 1, 9, 16],
 [0, 2, 5, 12, 11, 13, 9, 8, 10, 4, 3, 6, 15, 7, 1, 14, 16],
 [0, 13, 14, 12, 4, 1, 10, 11, 15, 6, 2, 8, 5, 3, 7, 9, 16],
 [0, 5, 8, 1, 6, 14, 2, 

In [190]:
def calculate_cost(sequence, time_matrix, distance_matrix, w1, w2):
    cost1 = 0
    cost2 = 0
    cost = 0

    for i in range(len(sequence) - 1):
        node1 = sequence[i]
        node2 = sequence[i + 1]

        cost1 += time_matrix.iloc[node1, node2]
        cost2 += distance_matrix.iloc[node1, node2]

        cost += (w1 * time_matrix.iloc[node1, node2]) + (w2 * distance_matrix.iloc[node1, node2])

    return pd.DataFrame({'Σt': [cost1], 'Σd': [cost2], 'w1*Σt+w2*Σd': [cost]})

In [191]:
cost_for_each_sequence = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2) for sequence in random_seq]
result_df = pd.concat(cost_for_each_sequence, ignore_index=True)

In [192]:
result_df

,Σt,Σd,w1*Σt+w2*Σd
0,5.351500,257.9113,131.631400
1,5.616528,268.6136,137.115064
2,5.644222,273.7264,139.685311
3,4.461556,209.8718,107.166678
4,5.701028,275.0805,140.390764
5,5.031444,240.9631,122.997272
6,5.313056,256.8799,131.096478
7,5.954333,284.3801,145.167217
8,4.599056,224.2182,114.408628
9,5.272917,250.8411,128.057008


## Genetic Algorithm implementation

In [193]:
def initialize_population(population_size):
    population = generate_random_sequences(population_size)
    return population


In [194]:
def tournament_selection(population, fitness_values, tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmin(tournament_fitness)]
        selected_parents.append(population[winner_index])
    return selected_parents

In [195]:
def ordered_crossover(parent1, parent2):
    print("Crossing : ", parent1, parent2)

    crossover_point1 = np.random.randint(1, len(parent1) - 2)
    crossover_point2 = np.random.randint(crossover_point1 + 1, len(parent1) - 1)

    child = [-1] * len(parent1)
    child[crossover_point1:crossover_point2] = parent1[crossover_point1:crossover_point2]

    remaining_indices = [i for i in range(1, len(parent1) - 1) if parent2[i] not in child[crossover_point1:crossover_point2]]
    child[:crossover_point1] = [parent2[i] for i in range(1, crossover_point1 + 1)]
    child[crossover_point2:] = [parent2[i] for i in range(crossover_point2, len(parent2) - 1)]

    # Ensure starting and ending nodes remain fixed
    child[0] = parent1[0]
    child[-1] = parent1[-1]
    print("  Result  :  ", child)

    return child


In [196]:
def shift_mutation(sequence):
    mutation_point1 = np.random.randint(1, len(sequence) - 2)
    mutation_point2 = np.random.randint(mutation_point1 + 1, len(sequence) - 1)

    mutated_sequence = sequence[:mutation_point1] + sequence[mutation_point2:] + sequence[mutation_point1:mutation_point2]

    # Ensure starting and ending nodes remain fixed
    mutated_sequence[0] = sequence[0]
    mutated_sequence[-1] = sequence[-1]

    print( " Original : ",sequence, " Mutated : ", mutated_sequence)
    return mutated_sequence

In [197]:
def genetic_algorithm(num_generations, population_size, tournament_size):
    population = initialize_population(population_size)
    for generation in range(num_generations):
        # Calculate fitness values for each individual in the population
        fitness_values = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0] for sequence in population]

        # Perform tournament selection to select parents
        parents = tournament_selection(population, fitness_values, tournament_size)
        # Perform ordered crossover to generate offspring
        offspring = []
        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                child1 = ordered_crossover(parents[i], parents[i + 1])
                child2 = ordered_crossover(parents[i + 1], parents[i])
                offspring.extend([child1, child2])

        # Perform shift mutation on the offspring
        mutated_offspring = [shift_mutation(child) for child in offspring]

        # Replace the old population with the new one
        population = mutated_offspring

    # Select the best individual from the final population
    best_sequence = min(population, key=lambda x: calculate_cost(x, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0])

    return best_sequence

In [198]:
best_sequence = genetic_algorithm(num_generations=5, population_size=20, tournament_size=5)
print("Best Sequence:", best_sequence)
# print("Best Cost:", calculate_cost(best_sequence, time_matrix, distance_matrix, w1, w2))

Crossing :  [0, 7, 14, 6, 12, 11, 15, 13, 5, 3, 2, 1, 4, 10, 8, 9, 16] [0, 13, 11, 6, 4, 10, 9, 12, 5, 8, 7, 1, 15, 14, 2, 3, 16]


NameError: name 'random' is not defined